In [3]:
import importlib

import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time, sleep
import argparse as ap
import sys
import csv
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
from fredapi import Fred

db = wrds.Connection()

Enter your WRDS username [dlflath23]:dflath19
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [4]:
def option_table(date):
    s = 'opprcd' + date[0:4]
    return s

def adj_date(secid, date):
    q1 = ("select secid, date, exdate "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date >= '%s'" % (option_table(date), secid, date) )
    df = db.raw_sql(q1)
    return df.date.unique()[0].strftime('%Y-%m-%d')

def get_secid(ticker, date):
    q1 = ("select secid, optionid, ticker, effect_date "
      "from optionm.optionmnames "
      "where ticker = '%s' and effect_date < '%s'" % (ticker, date) )
    df = db.raw_sql(q1)
    return df.sort_values(by=['effect_date']).iloc[-1].secid

def target_ex_date(start_date, days):
    date_1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = date_1 + datetime.timedelta(days=days)
    #date_string = end_date.strftime('%Y-%m-%d')
    return end_date.date()

def available_ex_dates(secid,date):
    q1 = ("select secid, date, exdate "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date = '%s'" % (option_table(date), secid, date) )
    df = db.raw_sql(q1)
    return df.exdate.unique()

def available_strikes(secid,date,ex_date):
    q1 = ("select secid, date, exdate, strike_price "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date = '%s' and exdate = '%s'" % (option_table(date), secid, date, ex_date) )
    df = db.raw_sql(q1) 
    strikes = df.strike_price.unique()
    return [x / 1000.00 for x in strikes]

def nearest(list, pivot):
    return min(list, key=lambda x: abs(x - pivot))

def one_day_pricing(secid,date,ex_date):
    q1 = ("select secid, date, symbol, optionid, exdate, strike_price, best_bid, best_offer "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date = '%s' and exdate = '%s'" % (option_table(date), secid, date, ex_date) )
    df = db.raw_sql(q1)
    return df

def one_contract_pricing(secid,date,ex_date,strike_price):
    q1 = ("select secid, date, symbol, optionid, exdate, strike_price, best_bid, best_offer "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date >= '%s' and exdate = '%s' and strike_price = %g" % (option_table(date), secid, date, ex_date, strike_price) )
    df = db.raw_sql(q1)
    return df

def hi_lo_avg(secid,date):
    q1 = ("select secid, date, high, low "
      "from optionm.secprd "
      "where secid = %g and date = '%s'" % (secid, date) )
    df = db.raw_sql(q1)
    return round((df.high + df.low)/2.00,2)

def nearest_strike(secid,date,delta,strikes):
    p = hi_lo_avg(secid, date)
    p_target = np.float64(p * delta)
    s = nearest(strikes,p_target)
    for i in xrange(3):
        if s > p * (delta + (1.00-delta)/2.00):
            p_target = np.float64(p * (delta - (1.00-delta)/2.00))
            s = nearest(strikes,p_target)
            break
    if s > p * (delta + (1.00-delta)/2.00):
        return None
    else:
        return s * 1000.00

def nearest_end_date(end_dates, end_date_date):
    date2 = end_date_date
    while True:
        date = nearest(end_dates,date2)
        if min(end_dates) > end_date_date:
            date = date2
            break
        if date <= end_date_date:
            break
        date2 = date2 - datetime.timedelta(days=1)
    return date

def average_ex_date(results):
    d=[]
    for t in results:
        d.append(results[t]['expiry']) 
    a = np.array(list(d), dtype='datetime64').view('i8').mean().astype('datetime64[D]')
    return pd.to_datetime(str(a))

def cost_final(ticker,start_date,end_date,delta,fwd,verbose=False):
    sec_id = get_secid(ticker, start_date)
    if verbose:
        print("Security ID: " + str(sec_id))
    start_date = adj_date(sec_id,start_date)
    if verbose:
        print("Adjusted Start Date: " + start_date)
    dates = available_ex_dates(sec_id, start_date)
    if verbose:
        print("Available ex dates: ")
        print(dates)
    target_ex = target_ex_date(start_date,fwd)
    if verbose:
        print("Target ex date: " + target_ex.strftime('%Y-%m-%d'))
    nearest_ex = nearest(dates,target_ex)
    if verbose:
        print("Nearest ex date: " + nearest_ex.strftime('%Y-%m-%d'))
    strikes = available_strikes(sec_id, start_date,nearest_ex)
    if verbose:
        print("Available strikes: ")
        print(strikes)
    hl = hi_lo_avg(sec_id,start_date)
    if verbose:
        print("Current price: " + str(hl)) 
        print("Target strike: " + str(round((hl * delta),2)))  
    strike = nearest_strike(sec_id,start_date,delta,strikes)
    if verbose:
        print("Closest strike: " + str(strike/1000.00))
        print('.............................')
        print('.............................')
    prices = one_contract_pricing(sec_id,start_date,nearest_ex,strike)
    start_bid = prices.iloc[0].best_bid
    start_ask = prices.iloc[0].best_offer
    cost = round((start_bid * 0.67) + (start_ask * 0.33),3)
    cost_date = prices.iloc[0].date
    if end_date is None:
        final_bid = prices.iloc[-1].best_bid
        final_ask = prices.iloc[-1].best_offer
        final = round((final_bid * 0.33) + (final_ask * 0.67),3)
        final_date = nearest_ex.strftime('%Y-%m-%d')
    else:
        end_dates = prices.date.unique()
        end_date_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
        closest_end_date = nearest_end_date(end_dates,end_date_date)
        prices_data = prices.loc[prices['date'] == closest_end_date]
        final_bid = prices_data.best_bid.iloc[0]
        final_ask = prices_data.best_offer.iloc[0]
        final = round((final_bid * 0.33) + (final_ask * 0.67),3)
        final_date = prices_data.date.iloc[0]
    s_yield = round(cost / (strike/1000.00*1.00),4)
    hp = (nearest_ex - cost_date).days*1.00/360.00
    a_yield = round(((1.00 + s_yield)**(1.00/hp)) - 1.00,4)
    d = OrderedDict()
    d["ticker"] = ticker
    d["trade_date"] = cost_date.strftime('%Y-%m-%d')
    d["expiry"] = nearest_ex.strftime('%Y-%m-%d')
    d["current"] = hl
    d["strike"] = strike/1000.00
    d["delta"] = round(strike/1000.00/hl - 1.00,4)
    d["c_date"] = cost_date.strftime('%Y-%m-%d')
    d["c_bid"] = start_bid
    d["c_ask"] = start_ask
    d["c_mid"] = cost
    d["f_date"] = str(final_date)
    d["f_bid"] = final_bid
    d["f_ask"] = final_ask
    d["f_mid"] = final
    d["gross_profit"] = (cost - final)
    d["s_yield"] = s_yield
    d["a_yield"] = a_yield
 
    return d

def calculate_number_of_contracts(starting_capital,results,max_pos_size):
    number_of_contracts = len(results.values())
    target_dollar_per_position = starting_capital / number_of_contracts
    for r in results.values():
        if target_dollar_per_position > (starting_capital * max_pos_size):  
            r["target_notional"] = (starting_capital * max_pos_size)
            contracts = round(starting_capital * max_pos_size/r["strike"]/100,0)
        else:
            r["target_notional"] = round(target_dollar_per_position,2)
            contracts = round(target_dollar_per_position/r["strike"]/100,0)
        if contracts < 1:
            r["contracts"] = 1
        else:
            r["contracts"] = contracts
        r["actual_notional"] = round(contracts * r["strike"] * 100, 2)
        r["total_profit"] = round(contracts * r["gross_profit"] * 100,2)
        r["ending_total"] = r["target_notional"] + r["total_profit"] 
    return results

def filter_for_min_yield(results,min_yield):
    remove = []
    for r in results:
        yld = results[r]["a_yield"]
        if yld < min_yield:
            #results.pop(r)
            remove.append(r)
    for p in remove:
        results.pop(p)
    return results

def filter_yield_rank(results,n):
    ranks = []
    for r in results:
        yld = results[r]["a_yield"]
        ranks.append((r,yld))
    sorted_by_second = sorted(ranks, key=lambda tup: tup[1])
    remove = []
    l = len(sorted_by_second)
    if l > n:
        for t in sorted_by_second[:(l-n)]:
            remove.append(t[0])
        for p in remove:
            results.pop(p)
    else:
        None
    return results

def total_period_profit(results):
    total = 0.00 
    for r in results.values():
        total = total + r["total_profit"]
    return round(total,2)

def get_fred_data():
    fred = Fred(api_key='5efb16a4685c34b5a642fa2f07ad97b7')
    d = fred.get_series('DTB3')
    df = pd.DataFrame(data=d).ffill()
    return df

fred_data = get_fred_data()

def add_interest(start_date,starting_capital,results,fred_data):
    start_date = np.datetime64(datetime.datetime.strptime(start_date, "%Y-%m-%d"))
    dates = fred_data.index.values
    date = nearest(dates,start_date)
    #print("Closest date: " + str(date)[:10])
    #print("Rate raw: " + str(fred_data.loc[date][0]))
    rate = fred_data.loc[date][0] / 100.00
    through_date = np.datetime64(average_ex_date(results))
    holding_period = (through_date - start_date).item().days*1.00/360.00
    interest_earned = starting_capital * rate * holding_period
    #print("Interest rate: " + str(round(rate,4)*100) + "%")
    #print("Interest earned: " + "$" + str(round(interest_earned,2)))
    return round(interest_earned,2)

def pretty_print(cf_data):
    print("Trade date:" + cf_data["trade_date"])
    print("Expiry date: " + cf_data["expiry"])
    print("Current Price: " + str(cf_data["current"]))
    print("Strike: " + str(cf_data["strike"]))
    print("Actual delta: " + str(cf_data["delta"] * 100) + "%")
    print("Start Date: " + cf_data["c_date"])
    print("Start Cost: " + str(cf_data["c_bid"]))
    print("Final Date: " + cf_data["f_date"])
    print("Final Bid: " + str(cf_data["f_bid"]))
    print("Final Ask: " + str(cf_data["f_ask"]))
    print("Final Mid: " + str(cf_data["f_mid"]))
    print("Simple Yield: " + str(cf_data["s_yield"] * 100) + "%")
    print("Annualized Yield: " + str(cf_data["a_yield"] * 100) + "%")
    
def confirm_bid_available(results,cf,start_date,start_date_stick):
    if cf["c_bid"] == 0.0:
        while True:
            start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            start_date = start_date + datetime.timedelta(days=1)
            away = start_date - start_date_stick
            if away.days >= 10:
                print(" -> No available trade dates within 10 days")
                break
            start_date = start_date.strftime('%Y-%m-%d')
            cf = cost_final(ticker,start_date,None,delta,fwd,verbose)
            ed = datetime.datetime.strptime(cf['expiry'], "%Y-%m-%d")
            if cf["c_bid"] > 0.0:
                if (datetime.datetime.strptime(cf['trade_date'], "%Y-%m-%d") - start_date).days > 10:
                    print(" -> Pass, trade date more than 10 days away")
                    break
                else:
                    final_cf = cf
                    break
    else:
        final_cf = cf
    return final_cf

def write_csv(start_date,results):
    print("      --> Done. Writing CSV file")

    with open('option_results_' + start_date + '.csv', 'w') as csvfile:
        fieldnames = results.values()[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for t in results:
            ticker = t
            data = results[t]
            writer.writerow(data)

def write_total_csv(total_pl):
    print("****** Writing Final CSV file ******")

    with open('total_pl.csv', 'w') as csvfile:
        w = csv.writer(csvfile)
        w.writerow(total_pl[0].keys())
        for r in total_pl:
            w.writerow(r.values())

In [5]:
def testing():
    ticker = 'AGP'
    start_date = '2005-02-20'
    end_date = None
    start_date_stick = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    delta = 0.9
    fwd = 40
    verbose = True

    cf = cost_final(ticker,start_date,end_date,delta,fwd,verbose)
    return cf
    #pretty_print(cf)
    if cf["c_bid"] == 0.0:
        print('.............................')
        print("No bid on this date, trying next date")
        print('.............................')
        while True:
            start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            start_date = start_date + datetime.timedelta(days=1)
            away = start_date - start_date_stick
            print("Looking ahead " + str(away.days) + " day(s)")
            if away.days >= 10:
                print("No bids within 10 days of target")
                break
            start_date = start_date.strftime('%Y-%m-%d')
            cf = cost_final(ticker,start_date,end_date,delta,fwd,verbose)
            if cf["c_bid"] > 0.0:
                return cf
                #pretty_print(cf)
                break

testing()

Security ID: 113135.0
Adjusted Start Date: 2005-02-22
Available ex dates: 
[datetime.date(2005, 3, 19) datetime.date(2005, 4, 16)
 datetime.date(2005, 6, 18) datetime.date(2005, 9, 17)]
Target ex date: 2005-04-03
Nearest ex date: 2005-04-16
Available strikes: 
[40.0, 45.0, 50.0]
Current price: 40.08
Target strike: 36.07


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [ ]:
#db.list_tables('optionm')

In [ ]:
#db.describe_table('compm', 'company')

In [5]:
def previous_quarter(date_string):
    ref = datetime.datetime.strptime(date_string, "%Y-%m-%d").date()
    if ref.month < 4:
        return datetime.date(ref.year - 1, 12, 31)
    elif ref.month < 7:
        return datetime.date(ref.year, 3, 31)
    elif ref.month < 10:
        return datetime.date(ref.year, 6, 30)
    return datetime.date(ref.year, 9, 30)

def top_n_mkt_cap_gvkeys(date_string,n,gvkeys):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate, gvkey,cshoq,prccq,cshoq*prccq as market_cap, curcdq "
            "from compm.fundq "
           "where cshoq>0 and prccq>0 and curcdq='USD' and datadate > '%s' and datadate <= '%s' and gvkey in (%s) "
         "order by market_cap desc "
        "limit %i") % (date2,date1,gvkeys_list,n)
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def gvkeys_for_filtered_security_type(date_string):
    year = date_string[0:4]
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    q1 = ("select gvkey,cyear,tpci,exchg,datadate "
            "from compm.secm "
           "where datadate > '%s' and datadate <= '%s' and tpci in ('0','F') and exchg in (11,12,14,19) ")% (date2,date1)
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def gvkeys_for_filtered_sectors(date_string,gvkeys):
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select gvkey,gsector,gind,loc "
            "from compm.company "
           "where gsector != '40' and gind != '151040' and gvkey in (%s) and loc='USA' ")% gvkeys_list
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def get_latest_prices(date_string,gvkeys):
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate,gvkey,prccd,tpci "
            "from compm.secd "
           "where datadate = '%s' and gvkey in (%s) and tpci in ('0','F') ") % (date_string,gvkeys_list)
    df = db.raw_sql(q1)
    while True:
        if df.gvkey.nunique() == 0:
            date = datetime.datetime.strptime(date_string, "%Y-%m-%d") - datetime.timedelta(days=1)
            date_string = date.strftime('%Y-%m-%d')
            q1 = ("select datadate,gvkey,prccd,tpci "
            "from compm.secd "
           "where datadate = '%s' and gvkey in (%s) and tpci in ('0','F') ") % (date_string,gvkeys_list)
            df = db.raw_sql(q1)
        else:
            break
    return df

def get_cheapest_n_tics(date_string,gvkeys,n):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    date3 = date2 - relativedelta(years=4)
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate,gvkey,tic,cusip,oiadpq,cshoq,prccq,pstkq, dlcq, dlttq, cheq,niq,ceqq "
            "from compm.fundq "
           "where datadate <= '%s' and datadate > '%s' and gvkey in (%s) ") % (date2,date3,gvkeys_list)
    df = db.raw_sql(q1)
    df["date"] = pd.to_datetime(df['datadate'], format='%Y-%m-%d')
    df.sort_values(by=["date"])
    df = df.groupby('gvkey').ffill()
    df['oiadpq_ttm'] = df.groupby('gvkey')['oiadpq'].apply(lambda x: x.ffill().rolling(window=4).sum())
    df['ni_ttm'] = df.groupby('gvkey')['niq'].apply(lambda x: x.ffill().rolling(window=4).sum())
    df['eqity_ttm'] = df.groupby('gvkey')['ceqq'].apply(lambda x: x.ffill().rolling(window=4).sum()/4)
    df['roe'] = df['ni_ttm'] / df['eqity_ttm']
    df['roe_3yr_avg'] = df.groupby('gvkey')['roe'].apply(lambda x: x.ffill().rolling(window=12).sum()/12)
    latest_prices = get_latest_prices(date_string,gvkeys)
    df = pd.merge(df, latest_prices, on = 'gvkey', how='left')
    df["market_cap"] = df["cshoq"] * df["prccd"]
    df["ent_value"] = df["market_cap"] + df["pstkq"] + df["dlcq"] + df["dlttq"] - df["cheq"]
    df["ev_ebit"] = (df["ent_value"] / df["oiadpq_ttm"])
    df["net_debt"] = df["dlcq"] + df["dlttq"] - df["cheq"]
    df["net_debt_ebit_ttm"] = df["net_debt"] / df['oiadpq_ttm']
    df = df.round({'market_cap': 2, 'ent_value': 2, 'ev_ebit' : 2, 'roe' : 4, 'roe_3yr_avg' : 4, 'net_debt_ebit_ttm' : 2})
    df = df.loc[df['ev_ebit'] > 0]
    df = df.sort_values(by=["roe_3yr_avg"],ascending=False)
    df = df.groupby('gvkey').apply(lambda g: g[g['date'] == g['date'].max()])
    df = df.loc[df['roe_3yr_avg'] >= 0.10]
    df = df.loc[df['net_debt_ebit_ttm'] <= 4.00]
    df = df.sort_values(by=["ev_ebit"])
    df = df.head(n)
    tics = [t.split('.')[0] for t in list(df.tic.unique())]
    
    #return df.filter(items=['tic','date','niq','ceqq','roe','roe_3yr_avg','ev_ebit','net_debt_ebit_ttm'])
    #return df
    return tics

def get_tics(date_string,n):
    gvkeys = gvkeys_for_filtered_security_type(date_string)
    sector_filtered = gvkeys_for_filtered_sectors(date_string,gvkeys)
    top_n_gvkeys = top_n_mkt_cap_gvkeys(date_string,1000,sector_filtered)
    d = get_cheapest_n_tics(date_string,top_n_gvkeys,n)
    return d

#get_tics('2005-01-01',100)

In [ ]:
rebal_data = [['2007-01-01','2007-01-31',
        ['XOM','CVX','COP','OXY','ECA','MRO','VLO','AET','APA','TLM','NUE','TECK','PCAR','SCCO','EOG','FCX','X','NBL','LEN','1086832D','CMI','ANDV','PTEN','VRX','FTO','CMC','STLD','RS','PDS','MEOH','HFC','CRS','MDC','FRO','CECO','LPX','UNT','OSGIQ','YRCW','EXP','CNW','SM','CLF','IDCC','MTH','SCHN','OLN','MLI','GMRRQ','KSWS']
        ],
        ['2007-02-01','2007-02-28',
        ['XOM','CVX','COP','OXY','ECA','MRO','VLO','AET','APA','TLM','NUE','TECK','PCAR','SCCO','EOG','FCX','X','NBL','LEN','1086832D','CMI','ANDV','PTEN','VRX','FTO','CMC','STLD','RS','PDS','MEOH','HFC','CRS','MDC','FRO','CECO','LPX','UNT','OSGIQ','YRCW','EXP','CNW','SM','CLF','IDCC','MTH','SCHN','OLN','MLI','GMRRQ','KSWS']
        ]
       ]

In [6]:
delta = 0.98
fwd = 40
verbose = False
starting_capital = 1000000
max_size = 0.05
min_yield = 0.10
target_n = 500
target_n_top_yields = 20

start_date = '2005-01-01'
stop_date = '2015-12-30'

total_pl = []
total_pl.append({'end_date' : start_date,
                'market_value' : starting_capital})

while True:
    if datetime.datetime.strptime(start_date, "%Y-%m-%d") > datetime.datetime.strptime(stop_date, "%Y-%m-%d"):
        break
    print("Getting tickers for " + start_date)
    tics = get_tics(start_date,target_n)
    if verbose:
        print(tics)
    results = {}
    count = 0.0
    print("   Calculating ")
    for t in tics:
        count = count + 1
        i = (count/len(tics))*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush() 
        start_date_stick = datetime.datetime.strptime(start_date,"%Y-%m-%d")
        try:
            if verbose:
                print("Calculating " + str(t))
            cf = cost_final(t,start_date,None,delta,fwd,verbose)
            if (datetime.datetime.strptime(cf['trade_date'], "%Y-%m-%d") - start_date_stick).days > 10:
                if verbose:
                    print(" -> Pass, trade date more than 10 days away")
                continue
            results[t] = confirm_bid_available(results,cf,start_date,start_date_stick)
        except:
            if verbose:
                print(" -> Ticker " + str(t) + " wasn't able to retrieve data")
            pass

    results = filter_for_min_yield(results,min_yield)
    
    results = filter_yield_rank(results,target_n_top_yields)
    
    if len(results.values()) > 1:  
        results = calculate_number_of_contracts(starting_capital,results,max_size)
        period_profit = total_period_profit(results)
        period_int = add_interest(start_date,starting_capital,results,fred_data)
        print("")
        print("   Starting Capital: " + "$" + str(round(starting_capital,2)))
        print("      Profit from options: " + "$" + str(period_profit))
        print("      Profit from interest: " + "$" + str(period_int))
        starting_capital = starting_capital + period_profit + period_int
        print("   Ending Capital: " + "$" + str(round(starting_capital,2)))
        write_csv(start_date,results)
        start_date = (average_ex_date(results) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        d = OrderedDict()
        d['end_date'] = start_date
        d['market_value'] = starting_capital
        total_pl.append(d)
    else:
        start_date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        
write_total_csv(total_pl)

Getting tickers for 2005-01-01
   Calculating 
100%
   Starting Capital: $1000000.0
      Profit from options: $10215.7
      Profit from interest: $2967.22
   Ending Capital: $1013182.92
      --> Done. Writing CSV file
Getting tickers for 2005-02-20
   Calculating 
100%
   Starting Capital: $1013182.92
      Profit from options: $-18902.2
      Profit from interest: $3962.67
   Ending Capital: $998243.39
      --> Done. Writing CSV file
Getting tickers for 2005-04-17
   Calculating 
100%
   Starting Capital: $998243.39
      Profit from options: $34244.6
      Profit from interest: $2686.94
   Ending Capital: $1035174.93
      --> Done. Writing CSV file
Getting tickers for 2005-05-22
   Calculating 
100%
   Starting Capital: $1035174.93
      Profit from options: $21432.8
      Profit from interest: $2259.27
   Ending Capital: $1058867.0
      --> Done. Writing CSV file
Getting tickers for 2005-06-19
   Calculating 
100%
   Starting Capital: $1058867.0
      Profit from options: $247

   Calculating 
100%
   Starting Capital: $1505375.81
      Profit from options: $-36747.2
      Profit from interest: $2544.92
   Ending Capital: $1471173.53
      --> Done. Writing CSV file
Getting tickers for 2008-06-22
   Calculating 
100%
   Starting Capital: $1471173.53
      Profit from options: $-22274.4
      Profit from interest: $2419.26
   Ending Capital: $1451318.39
      --> Done. Writing CSV file
Getting tickers for 2008-07-25
   Calculating 
100%
   Starting Capital: $1451318.39
      Profit from options: $90707.3
      Profit from interest: $3929.44
   Ending Capital: $1545955.13
      --> Done. Writing CSV file
Getting tickers for 2008-09-21
   Calculating 
100%
   Starting Capital: $1545955.13
      Profit from options: $-440252.5
      Profit from interest: $1551.11
   Ending Capital: $1107253.74
      --> Done. Writing CSV file
Getting tickers for 2008-10-20
   Calculating 
100%
   Starting Capital: $1107253.74
      Profit from options: $-154078.3
      Profit fro

   Calculating 
100%Getting tickers for 2011-11-25
   Calculating 
100%
   Starting Capital: $1735625.47
      Profit from options: $78406.6
      Profit from interest: $54.96
   Ending Capital: $1814087.03
      --> Done. Writing CSV file
Getting tickers for 2012-01-22
   Calculating 
100%
   Starting Capital: $1814087.03
      Profit from options: $91489.5
      Profit from interest: $58.45
   Ending Capital: $1905634.98
      --> Done. Writing CSV file
Getting tickers for 2012-02-21
   Calculating 
100%
   Starting Capital: $1905634.98
      Profit from options: $-8445.0
      Profit from interest: $105.87
   Ending Capital: $1897295.85
      --> Done. Writing CSV file
Getting tickers for 2012-03-18
   Calculating 
100%
   Starting Capital: $1897295.85
      Profit from options: $-51649.2
      Profit from interest: $179.19
   Ending Capital: $1845825.84
      --> Done. Writing CSV file
Getting tickers for 2012-04-22
   Calculating 
100%
   Starting Capital: $1845825.84
      Profit

   Calculating 
100%
   Starting Capital: $2089024.93
      Profit from options: $-100705.3
      Profit from interest: $19.73
   Ending Capital: $1988339.36
      --> Done. Writing CSV file
Getting tickers for 2015-07-27
   Calculating 
100%
   Starting Capital: $1988339.36
      Profit from options: $-22301.3
      Profit from interest: $115.99
   Ending Capital: $1966154.05
      --> Done. Writing CSV file
Getting tickers for 2015-09-08
   Calculating 
100%
   Starting Capital: $1966154.05
      Profit from options: $18513.9
      Profit from interest: $124.52
   Ending Capital: $1984792.47
      --> Done. Writing CSV file
Getting tickers for 2015-10-17
   Calculating 
100%
   Starting Capital: $1984792.47
      Profit from options: $-615.2
      Profit from interest: $19.85
   Ending Capital: $1984197.12
      --> Done. Writing CSV file
Getting tickers for 2015-11-23
   Calculating 
100%
   Starting Capital: $1984197.12
      Profit from options: $20029.3
      Profit from interest